# Amortisierte Analyse


Nicht immer ist die Angabe der worst-case Komplexität hilfreich. Was ist nämlich, wenn der worst-case nur sehr selten eintritt, aber in den meisten Fällen eine wesentlich bessere Laufzeit erzielt wird? Dies ist z.B. beim Hinzufügen von Elementen in ein dynamisches Array der Fall. In den meisten Fällen geht dies problemlos mit einem Aufwand von $\mathcal{O}(1)$, doch ist das Array nicht lang genug, so müssen alle Werte in ein neues Array kopiert werden, was zu einem Aufwand von $\mathcal{O}(n)$ führt. Dadurch liegt die worst-case Komplexität bei $\mathcal{O}(n)$, was zunächst ziemlich schlecht zu sein scheint, da andere Datenstrukturen dies (selbst im worst-case) in $\mathcal{O}(1)$ können. Um hier eine genaue Aussage treffen zu können, bedarf es der amortisierten Analyse.

Die amortisierte Analyse (amortized analysis) berechnet die mittleren Kosten über eine Folge von Operationen, in der viele dieser Operationen billig sind und nur wenige teuer in Bezug auf deren Beitrag zur Gesamtzeit.

Wir betrachten eine Folge von $n$ (durchaus verschiedener) Operationen und mitteln die Zeit. Dabei kommt der worst case sehr selten vor. Die amortisierte Analyse bestimmt die mittlere Performance jeder Operation im schlechtesten Fall.

## Aggregat-Analyse

Bei der Aggregat-Analyse gilt:

$$T(n)_{amort} = \frac{\text{Summe der Kosten aller $n$ Operationen der Folge}}{n}$$

Die Zeitaufwände der $n$ Operationen der betrachteten Operationenfolge wird durch die Anzahl $n$ der Operationen geteilt.


### Dynamisches Array

Bei einem dynamischen Array wird zunächst ein Array der Größe $1$ initialisiert. Sobald der Platz nicht mehr ausreicht und man das $n+1$. Element einfügen möchte, wird ein neues Array mit doppelter Länge erstellt. Die Werte des ursprünglichen Feldes werden in das neue hineinkopiert. Sollte dieses Array nicht mehr ausreichen, so wird es durch ein neues doppelter Länge ersetzt usw.

Die Operation "Einfügen eines Elements in ein Array" wird also sehr oft mit $\mathcal{O}(1)$ zu bewerkstelligen sein und nur sehr selten $\mathcal{O}(n)$ erfordern.

Die amortisierten Kosten betragen:

$$
\begin{align*}
T(n)_{amort} & = \frac{(n-1)\cdot\mathcal{O}(1)+1\cdot\mathcal{O}(n)}{n}=\mathcal{O}(1)
\end{align*}
$$

#### Alternative amortisierte Analyse

$$\begin{align*}
T(n)_{gesamt} & = n + 1 + \dotsc + 1 + \frac{n}{2} + 1 + \dotsc + 1 + \frac{n}{4} + 1 + \dotsc + 1 + \frac{n}{8} + \dotsc + 1 \\
 & = \sum_{i=0}^{\infty} n \cdot \left(\frac{1}{2} \right)^i  + (n - \log_2 n) \cdot 1 \\
 & = n \cdot \sum_{i=0}^{\infty} \left(\frac{1}{2}\right)^i + (n - \log_2 n) \cdot 1 \\
\end{align*}
$$

Da es sich bei der Summe um eine geometrische Reihe handelt, kann sie aufgelöst werden.

$$
\begin{align*}
T(n)_{gesamt} & = n \cdot \frac{1}{1-\frac{1}{2}} + (n - \log_2 n) \cdot 1 \\
 & = 2n + (n - \log_2 n) \cdot 1 \\
 & = 2n + \mathcal{O}(n) \\
 & \in \mathcal{O}(n)
\end{align*}
$$

$$
\begin{align*}
T(n)_{amort} & = \frac{\mathcal{O}(n)}{n} \\
 & \in \mathcal{O}(1)
\end{align*}
$$
<!-- <div style="text-align: right; font-size: 24px;">&#9633;</div> --> 

Damit ergeben sich für die Insert-Opertion bei einem dynamischen Array amortisierte Kosten von $\mathcal{O}(1)$.

### Binärzähler

Situation: $n$ Inkrementierungen eines $k$-Bit-Binärzählers mit Anfangswert 0, siehe auch [HPI teaching](https://hpi.de/friedrich/teaching/units/amortisierte-analyse.html)

Array: $A[0 \dotsc k-1], A[i] \in \{0,1\},$

$$x = \sum_{i=0}^{k-1} A[i] \cdot 2^i$$

Aufwand: akkumulierte Anzahl der "Bit-Klappungen"

Worst case: Alle $k$ Bits werden invertiert, da sie von 1 auf 0 wechseln. Dies ist beim Übergang von $2^i - 1$ zu $2^i$ der Fall. Folglich beträgt der Aufwand je Inkrementierungsschritt im schlechtesten Fall $\mathcal{O}(k) = \mathcal{O}(\log n)$. Dies ist zwar korrekt, aber zu grob.

In [2]:
import pandas as pd


def increment_cost(n):
    if n == 0:
        return 0
    return n ^ (n-1)


def total_costs(bin_strings):
    if len(bin_strings) == 0:
        return []
    total_costs_lst = total_costs(bin_strings[:-1])
    if len(total_costs_lst) == 0:
        total_cost = 0
    else:
        total_cost = total_costs_lst[len(total_costs_lst)-1]
    for digit in bin_strings[len(bin_strings)-1]:
        if digit == "1":
            total_cost += 1
    return total_costs_lst + [total_cost]

    
A_cost = []
max_n = 25
inc_costs = []

for i in range(0, max_n):
    inc_costs.append("{0:b}".format(increment_cost(i)))

for i in range(0, 7):
    A_cost.append(list())
    for j in range(0, max_n):
        A_cost[i].append(0)

for n in range(0, max_n):
    inc_cost = "{0:b}".format(increment_cost(n))
    for i in reversed(range(0, len(inc_cost))):
        A_cost[i][n] = inc_cost[i]
        
        
table = pd.DataFrame({'A[6]': pd.Series(A_cost[6]),
                     'A[5]': pd.Series(A_cost[5]),
                     'A[4]': pd.Series(A_cost[4]),
                     'A[3]': pd.Series(A_cost[3]),
                     'A[2]': pd.Series(A_cost[2]),
                     'A[1]': pd.Series(A_cost[1]),
                     'A[0]': pd.Series(A_cost[0]),
                    'Total costs': pd.Series(total_costs(inc_costs))}, 
                   columns=['A[6]', 'A[5]', 'A[4]', 'A[3]', 'A[2]', 'A[1]', 'A[0]', 'Total costs'])

print(table)

    A[6]  A[5] A[4] A[3] A[2] A[1] A[0]  Total costs
0      0     0    0    0    0    0    0            0
1      0     0    0    0    0    0    1            1
2      0     0    0    0    0    1    1            3
3      0     0    0    0    0    0    1            4
4      0     0    0    0    1    1    1            7
5      0     0    0    0    0    0    1            8
6      0     0    0    0    0    1    1           10
7      0     0    0    0    0    0    1           11
8      0     0    0    1    1    1    1           15
9      0     0    0    0    0    0    1           16
10     0     0    0    0    0    1    1           18
11     0     0    0    0    0    0    1           19
12     0     0    0    0    1    1    1           22
13     0     0    0    0    0    0    1           23
14     0     0    0    0    0    1    1           25
15     0     0    0    0    0    0    1           26
16     0     0    1    1    1    1    1           31
17     0     0    0    0    0    0    1       

Die Tabelle zeigt, wann ein Bit invertiert werden muss. In diesem Fall steht eine 1 an dem entsprechenden Index. Man kann beobachten, dass $A[0]$ $n$-mal invertiert wird, $A[1]$ wird in jedem zweiten Schritt invertiert, also $\frac{n}{2}$-mal. Verallgemeinert kann man sagen, dass $A[i]$ $\frac{n}{2^i}$-mal invertiert werden muss. Für die Gesamtkosten ergibt sich also:

$$T(n)_{\text{gesamt}} = \sum_{i=0}^{k-1} \frac{n}{2^i} = n \cdot \sum_{i=0}^{k-1} \frac{1}{2^i} < n \cdot \sum_{i=0}^{\infty} \frac{1}{2^i} = 2n \in \mathcal{O}(n)$$

$$T(n) = \frac{\mathcal{O}(n)}{n} \in \mathcal{O}(1)$$
<div style="text-align: right; font-size: 24px;">&#9633;</div>

Es ergibt sich also ein amortisierter Aufwand von $\mathcal{O}(1)$ für die Inkrement-Operation.

Die Aggregat Methode ist die einfachste Methode zur amortisierten Analyse, jedoch lassen sich komplexere Algorithmen mit ihr nicht lösen.

## Accounting-Methode

Bei der Accounting-Methode gibt es ein Bankkonto, auf welches man Guthaben laden kann. Dieses Guthaben kann man sich als Münzen vorstellen, die eingezahlt werden, wenn eine Operation billig ist, d.h. mit sehr geringem Zeitaufwand ausgeführt wird. Bei Operationen mit hohen Kosten (großer Zeitaufwand) besteht die Möglichkeit, vorhandenes Guthaben vom Konto zu nehmen und damit die Operation zu "bezahlen". Der Betrag auf dem Konto darf dabei nicht negativ werden, man möchte nämlich zeigen, dass die tatsächlichen Kosten $\leqslant$ amortisierte Kosten sind.

### Dynamisches Array

Hier wird folgendermaßen vorgegangen:

- wenn eine Insertion-Operation keine Verdopplung verursacht, zahlt man eine Münze mit dem Wert $\mathcal{O}(1)$ in das Konto ein.
- wenn eine Insertion-Operation eine Verdopplung verursacht, so wurden seit der letzten Verdopplung $\frac{n}{2}$ Elemente eingefügt. $\frac{n}{2}$ Münzen können nun verwendet werden um die $\mathcal{O}(n)$ Operation zu bezahlen.

<img src="img/table-doubling-accounting-method.png" alt="Drawing" style="width: 400px;"/>

- amortisierte Kosten für eine Verdopplung: $\mathcal{O}(n) - c \cdot \frac{n}{2} = 0$ für ein $c$, das groß genug ist. (In dem Fall $c=2$)
- da $c$ eine Konstante ist, gilt dass die amortisierten Kosten für eine Insert-Operation $1 + c \in \mathcal{O}(1)$ sind.


## Potenzial-Methode (Potenzialfunktionsmethode)

Bei der Potenzial-Methode, bzw. beim Beweis mit Potenzialfunktion, wird eine Funktion definiert, die von einer Datenstruktur in einem bestimmten Zustand auf eine reelle nicht-negative Zahl abbildet. Diesen Wert bezeichnet man als Potenzial. Dieses Konzept ähnelt der Accounting Methode.

__Definition 4.2__ Es wird eine Datenstruktur zum Zeitpunkt $i$ als $D_i$ betrachtet. Zu definieren ist also eine Potenzialfunktion $\Phi : D_i \rightarrow \mathbb{R^+_0}$. Die tatsächlichen Kosten einer Operation werden $c_i$ bezeichnet. Für die amortisierten Kosten $\hat{c}_i$ ergibt sich folgende Gleichung:

$$
\hat{c}_i = c_i + \Delta \Phi(D_i) = c_i + \Phi(D_i) - \Phi(D_{i-1})
$$

Die amortisierten Kosten einer Operation sind die Summe aus den tatsächlichen Kosten dieser Operation und der Veränderung der Potenzialfunktion, die durch diese Operation verursacht wird. Die Veränderung der Potenzialfunktion ist gleich der Differenz aus $\Phi$ zum Zeitpunkt $i$ und $\Phi$ zum Zeitpunkt $i-1$.

Intuitiv kann man sagen, dass die Potenzialfunktion angeben soll, wie labil der aktuelle Zustand der Datenstruktur gegenüber teure Operationen ist, d.h. wie nah die nächste teure Operation ist.

Aus der Gleichung für die amortisierten Kosten $\hat{c}_i$ lässt sich eine Gleichung für die amortisierten Kosten aller Operationen von 1 bis $n$ herleiten:

__Lemma 4.3__
$$
\begin{align*}
\sum_{i=1}^n \hat{c}_i &= \sum_{i=1}^n c_i + \sum_{i=1}^n (\Phi(D_i) - \Phi(D_{i-1})) \\
 &= \sum_{i=1}^n c_i + (\Phi(D_1) - \Phi(D_0) + \Phi(D_2) - \Phi(D_1) + \dotsc + \Phi(D_n) - \Phi(D_{n-1})) \\
\end{align*}
$$

Bei der Teleskopsumme $\sum \Phi(D_i) - \Phi(D_{i-1})$ kürzen sich alle Terme außer $\Phi(D_n)$ und $\Phi(D_0)$. Somit ergibt sich:

__Korollar 4.4__
$$
\begin{align*}
\sum_{i=1}^n \hat{c}_i &= \sum_{i=1}^n c_i + \Phi(D_n) - \Phi(D_0) \\
\sum \text{amortisierte Kosten} &= \sum \text{tatsächliche Kosten} + \Phi(\text{finale Datenstruktur}) - \Phi(\text{initiale Datenstruktur})
\end{align*}
$$


### Dynamische Arrays

Bei dynamischen Arrays, deren Größe sich verdoppelt, wenn sie voll sind, lässt sich beispielsweise folgende Potenzialfunktion aufstellen:

$$
\Phi(D_i) = 2n-m
$$

$n$ ist dabei die Anzahl der eigentlichen Elemente im dynamischen Array und $m$ ist die Anzahl der allozierten Speicherplätze. Direkt nach einer Verdopplungsoperation ist die Hälfte der Speicherplätze mit Elementen gefüllt, also $n = \frac{m}{2}$. Damit ist 

$$
\Phi(D_i) = 2n-m = 2 \cdot \frac{m}{2} - m = 0
$$

$\Phi(D_i) = 0$ bedeutet, dass die Datenstruktur weit entfernt von der nächsten tueren Operation ist. Kurz vor der Verdopplungsoperation beträgt $n = m$, also ist 

$$
\Phi(D_i) = 2n - m = 2m - m = m
$$

Nun kann sowohl für die billige, als auch für die teure Operation bewiesen werden, dass die amortisierten Kosten in beiden Fällen $\mathcal{O}(1)$ betragen.

##### Pushback Operation ohne Speicherallokation

$$
\begin{align*}
\hat{c}_i &= c_i + \Delta \Phi(D_i) \\
 &= c_i + \Phi(D_i) - \Phi(D_{i-1}) \\
 &= 1 + 2(n+1) - m - (2n - m) \\
 &= 1 + 2 \\
 &= 3 \\
 &\in \mathcal{O}(1)
\end{align*}
$$

##### Pushback Operation mit Speicherallokation

In diesem Fall ist das momentane Array voll und es gilt $n = m$:

$$
\begin{align*}
\hat{c}_i &= c_i + \Delta \Phi(D_i) \\
 &= c_i + \Phi(D_i) - \Phi(D_{i-1}) \\
 &= n + 1 + 2(n + 1) - 2m - (2n - m) \\
 &= n + 1 + 2 - m \\
 &= n + 1 + 2 - n \\
 &= 3 \\
 &\in \mathcal{O}(1)
\end{align*}
$$
<div style="text-align: right; font-size: 24px;">&#9633;</div>

### Binärzähler

Beim Binärzahler könnte man zunächst intuitiv die Anzahl der hinterneinander folgenden Bits, die 1 sind, von hinten, als Potenzialfunktion nehmen. Demnach hätte 01001000 eine kleines Potenzial, ist also weit weg von teuren Operationen, 01111111 hätte demnach ein großes Potenzial, was auch zu stimmen scheint, da die nächste Operation zeitaufwendig sein wird. Betrachtet man aber beispielsweise 11111110, so wäre das Potenzial 0. Eine teure Operation ist also vermeindlich weit entfernt. Jedoch wird die übernächste Operation teuer. Die Anzahl der hinterneinander folgenden 1-Bits von hinten, scheint also keine passende Potenzialfuktion zu sein.

Stattdessen ist die Gesamtzahl der 1-Bits der Binärzahl aussagekräftiger. Man spricht auch vom Hamming-Gewicht (hamming weight) der Binärzahl.

$\text{hamming_weight}(01001000) = 2$, $\text{hamming_weight}(01111111) = 7$, $\text{hamming_weight}(11111110) = 7$.

__Definition 4.5__ Das Hamming-Gewicht einer Zeichenkette ist die Anzahl der vom Nullzeichen des verwendeten Alphabets verschiedenen Zeichen.

Auch hier kann gezeigt werden, dass die amortisierten Kosten für eine Inkrementoperation $\mathcal{O}(1)$ sind.

$$
\hat{c}_i = c_i + \Delta \Phi(D_i)
$$

Bei einem Inkrement einer Binärzahl werden alle aufeinanderfolgenden Bits, die 1 sind, zu 0 invertiert und das erste Bit von rechts, das 0 ist, wird zu 1 invertiert. Damit ergeben sich folgende Kosten $c$ für eine Binärzahl mit $t$ aufeinnanderfolgenden 1-Bits von hinten: $c = t + 1$

$$
\begin{align*}
\hat{c}_i &= t + 1 + \Delta \Phi(D_i) \\
 &= t + 1 + \Phi(D_i) - \Phi(D_{i-1}) \\
\end{align*}
$$

Das Potenzial $\Phi(D_{i-1})$ vor der Inkrementoperation ist $\text{hamming_weight}(b)$. Durch die Inkrementoperation werden $t$ aufeinanderfolgende 1-Bits zu 0, wodurch sich das Hamming-Gewicht um $t$ verkleinert. Das erste 0-Bit von hinten wird zu 1, dadurch vergrößert sich das Hamming-Gewicht um 1. Das Potenzial $\Phi(D_i)$ nach der Inkrementoperation beträgt demnach $\text{hamming_weight}(b) - t + 1$.

$$
\begin{align*}
\hat{c}_i &= t + 1 + \Delta \Phi(D_i) \\
 &= t + 1 + (\text{hamming_weight}(b) - t + 1) - (\text{hamming_weight}(b)) \\
 &= t + 1 - t + 1 \\
 &= 2 \\
 &\in \mathcal{O}(1)
\end{align*}
$$
<div style="text-align: right; font-size: 24px;">&#9633;</div>